In [33]:
try:
    import telegram
    import selenium
except:
    !pip install python-telegram-bot
    !pip install selenium

from telegram.ext import Updater, CommandHandler, MessageHandler, Filters

import logging
import re
import statistics
import time
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
                    level=logging.INFO)
logger = logging.getLogger(__name__)

f = open("token.txt", "r")
token = f.readline().split(" ")[-1]

driver_loc = r"C:\Users\Kai Jing\Desktop\KJ's Online Sync\Selenium\chromedriver.exe"
url = 'https://www2.sgx.com/securities/equities/'

def extract_stock(stock_codes=[]):
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--disable-gpu")
    
    driver = webdriver.Chrome(executable_path=driver_loc,options=chrome_options)
    n_seconds = 10
    
    all_info = ""
    
    for stock_code in stock_codes:
        try:
            driver.get(url+stock_code)

            price = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[2]/div[1]/div[1]')))
            stock_price = float(price.text)

            name_field = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[1]/h3'))).text
            co_name = name_field.split("(")[0].strip()

            time_retrieved = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[2]/div[1]/div[3]'))).text
            
            valuation_tab = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/sgx-state-tabs/div/div/span[2]')))
            driver.execute_script("arguments[0].click();", valuation_tab)

            pb_ratio = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[2]/sgx-content-table/div/table/tbody/tr[1]/td[1]'))).text
            pe_ratio = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[2]/sgx-content-table/div/table/tbody/tr[3]/td[2]'))).text
            
            dividend_tab = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/sgx-state-tabs/div/div/span[4]')))
            driver.execute_script("arguments[0].click();", dividend_tab)
            
            payment_date = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[4]/widget-corporate-actions/sgx-content-table/div/table/tbody/tr[1]/td[3]'))).text
            dividend_details = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[4]/widget-corporate-actions/sgx-content-table/div/table/tbody/tr[1]/td[4]'))).text
            
            driver.get("https://sg.finance.yahoo.com/quote/" + stock_code + ".SI")
            
            time.sleep(3)
            
            eps = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="quote-summary"]/div[2]/table/tbody/tr[4]/td[2]'))).text

            historical_data_tab = WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="quote-nav"]/ul/li[5]/a')))
            driver.execute_script("arguments[0].click();", historical_data_tab)
            
            time.sleep(3)
            
            historical_prices = []

            for i in range(25):
                if len(historical_prices) == 20:
                    break

                try:
                    xpath = '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr['+str(i+1)+']/td[5]'
                    closing_price = round(float(WebDriverWait(driver,n_seconds).until(EC.element_to_be_clickable((By.XPATH, xpath))).text),2)
                    historical_prices.append(closing_price)
                except:
                    continue
            
            moving_avg = np.mean(historical_prices)
            std_dev = statistics.stdev(historical_prices)

            if (stock_price >= moving_avg + 2*std_dev):
                mkt_sentiment = "Bullish"
            elif (stock_price <= moving_avg - 2*std_dev):
                mkt_sentiment = "Bearish"
            else:
                mkt_sentiment = "Neutral"
            
            all_info += co_name + " (" + stock_code + ") - Price: S$" + "%.2f" % stock_price + "\n"
            all_info += "Latest dividend (to be) paid on " + payment_date + " [" + dividend_details + "]\n"
            all_info += "P/B Ratio: " + pb_ratio + ", P/E Ratio: " + pe_ratio + ", EPS (TTM): " + eps
            all_info += ", Market Sentiment: " + mkt_sentiment + "\n" + "*" + time_retrieved + "\n\n"
        except:
            all_info += stock_code + " is an invalid stock code!\n\n"
    
        
    all_info += "**Market sentiments are determined using Bollinger Bands over a window of 20 days"
    
    driver.quit()
    
    return all_info
    
def sti():
    driver = webdriver.Chrome(executable_path=driver_loc)
    driver.get("https://en.wikipedia.org/wiki/Straits_Times_Index")

    all_codes = []

    for i in range(30):
        xpath = '//*[@id="mw-content-text"]/div/table[4]/tbody/tr['+str(i+1)+']/td[1]/a[2]'
        elem = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,xpath)))
        all_codes.append(elem.text)

    driver.quit()
    
    return extract_stock(all_codes)
    
    
def get_sti(update, context):
    update.message.reply_text("Please give me a moment while I retrieve the information...")
    info = sti()
    
    if len(info) > 4096:
        info_list = info.split("\n\n")
    
        reply = ""

        for item in info_list:
            if len(reply) + len(item) + 2 <= 4096:
                reply += item + "\n\n"
            else:
                update.message.reply_text(reply)
                reply = item + "\n\n"
        update.message.reply_text(reply)
        
    else:
        update.message.reply_text(info)
    
    
def get_stock(update, context):
    if " " in update.message.text:
        stock_code = update.message.text.split(" ")[-1].upper()
    else:
        update.message.reply_text("Please pass me the stock code together with the command in this format:\n"+
                                  "/stock <stock code>\n\n"+
                                  "Multiple stock codes can be passed in this format:\n"+
                                  "/stocks <stock1, stock2, ...>")
    
    update.message.reply_text("Please give me a moment while I retrieve the information...")
    stock_info = extract_stock([stock_code])
    update.message.reply_text(stock_info)
    
        
def get_stocks(update, context):
    if " " in update.message.text and "," in update.message.text:
        all_stocks = [item.strip().upper() for item in update.message.text.split(" ",1)[-1].split(",")]
    else:
        update.message.reply_text("Please pass me the stock code together with the command in this format:\n"+
                                  "/stock <stock code>\n\n"+
                                  "Multiple stock codes can be passed in this format:\n"+
                                  "/stocks <stock1, stock2, ...>")
    
    update.message.reply_text("Please give me a moment while I retrieve the information...")
    stock_info = extract_stock(all_stocks)
    update.message.reply_text(stock_info)


def start(update, context):
    """Send a message when the command /start is issued."""
    update.message.reply_text('Hi!')

def help(update, context):
    """Send a message when the command /help is issued."""
    update.message.reply_text("The following commands are available:\n"+
                              "/start               - Starts the service\n"+
                              "/help                - Provides help for available actions\n"+
                              "/stock <stock code>  - Returns stock summary of given stock code\n"+
                              "/stocks <stock1, stock2, ...>  - Returns stock summary of given stock codes\n"+
                              "/sti                 - Returns stock summary of all STI stocks\n"+
                              "/exit                - Exits bot operations")

def echo(update, context):
    """Echo the user message."""
    update.message.reply_text(update.message.text)
    
def message_handler(update, context):
    response = update.message.text
    
    if response[0] == "/":
        update.message.reply_text("The following commands are available:\n"+
                                  "/start               - Starts the service\n"+
                                  "/help                - Provides help for available actions\n"+
                                  "/stock <stock code>  - Returns stock summary of given stock code\n"+
                                  "/stocks <stock1, stock2, ...>  - Returns stock summary of given stock codes\n"+
                                  "/sti                 - Returns stock summary of all STI stocks\n"+
                                  "/exit                - Exits bot operations")
    else:
        update.message.reply_text("How can I help you today?")

def error(update, context):
    """Log Errors caused by Updates."""
    logger.warning('Update "%s" caused error "%s"', update, context.error)
    
def exit(update, context):
    while True:
        response = update.message.text
        if response == "/start":
            update.message.reply_text("Welcome back!")
            break
        time.sleep(3)
    
def main():
    """Start the bot."""
    # Create the Updater and pass it your bot's token.
    # Make sure to set use_context=True to use the new context based callbacks
    # Post version 12 this will no longer be necessary
    updater = Updater(token, use_context=True)
    
    # Get the dispatcher to register handlers
    dp = updater.dispatcher

    # on different commands - answer in Telegram
    dp.add_handler(CommandHandler("start", start))
    dp.add_handler(CommandHandler("help", help))
    dp.add_handler(CommandHandler("echo", echo))
    dp.add_handler(CommandHandler("stock", get_stock))
    dp.add_handler(CommandHandler("stocks", get_stocks))
    dp.add_handler(CommandHandler("sti", get_sti))
    dp.add_handler(CommandHandler("exit", exit))

    # on noncommand i.e message - echo the message on Telegram
    dp.add_handler(MessageHandler(Filters.text, message_handler))

    # log all errors
    dp.add_error_handler(error)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()

if __name__ == '__main__':
    main()

2020-06-11 12:02:47,972 - telegram.ext.updater - INFO - Received signal 2 (SIGINT), stopping...


In [74]:
stock_code = 'U11'

driver = webdriver.Chrome(executable_path=driver_loc)
driver.get(url+stock_code)

price = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[2]/div[1]/div[1]')))
stock_price = "%.2f" % float(price.text)

name_field = WebDriverWait(driver, 5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[1]/h3'))).text
co_name = name_field.split("(")[0].strip()

time_retrieved = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-header/div[1]/div[2]/div[1]/div[3]'))).text

valuation_tab = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/sgx-state-tabs/div/div/span[2]')))
driver.execute_script("arguments[0].click();", valuation_tab)

pb_ratio = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[2]/sgx-content-table/div/table/tbody/tr[1]/td[1]'))).text
pe_ratio = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[2]/sgx-content-table/div/table/tbody/tr[3]/td[2]'))).text

dividend_tab = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/sgx-state-tabs/div/div/span[4]')))
driver.execute_script("arguments[0].click();", dividend_tab)

payment_date = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[4]/widget-corporate-actions/sgx-content-table/div/table/tbody/tr[1]/td[3]'))).text
dividend_details = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-stocks-company-data/div[4]/widget-corporate-actions/sgx-content-table/div/table/tbody/tr[1]/td[4]'))).text

driver.get("https://sg.finance.yahoo.com/quote/" + stock_code + ".SI")

eps = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH,'//*[@id="quote-summary"]/div[2]/table/tbody/tr[4]/td[2]'))).text

historical_data_tab = WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, '//*[@id="quote-nav"]/ul/li[5]/a')))
driver.execute_script("arguments[0].click();", historical_data_tab)

historical_prices = []

for i in range(25):
    if len(historical_prices) == 20:
        break
        
    try:
        xpath = '//*[@id="Col1-1-HistoricalDataTable-Proxy"]/section/div[2]/table/tbody/tr['+str(i+1)+']/td[5]'
        closing_price = round(float(WebDriverWait(driver,10).until(EC.presence_of_element_located((By.XPATH, xpath))).text),2)
        historical_prices.append(closing_price)
    except:
        continue

driver.quit()
        
moving_avg = np.mean(historical_prices)
std_dev = statistics.stdev(historical_prices)

if (stock_price >= moving_avg + 2*std_dev):
    mkt_sentiment = "Bullish"
elif (stock_price <= moving_avg - 2*std_dev):
    mkt_sentiment = "Bearish"
else:
    mkt_sentiment = "Neutral"

print(co_name + " (" + stock_code + ") - Price: S$" + str(round(stock_price,2)) + "\n" + 
      "Latest dividend (to be) paid on " + payment_date + " [" + dividend_details + "]\n" +
      "P/B Ratio: " + pb_ratio + ", P/E Ratio: " + pe_ratio + ", EPS (TTM): " + eps + 
      ", Market Sentiment: " + mkt_sentiment + "\n" +
      "*" + time_retrieved + "\n\n")

TypeError: '<=' not supported between instances of 'numpy.ndarray' and 'str'

In [21]:
driver = webdriver.Chrome(executable_path=driver_loc)
driver.get("https://www2.sgx.com/indices/products/sti")

WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,'//*[@id="gdpr-banner"]/div/button'))).click()
driver.execute_script("window.scrollTo(0, 0.6*document.body.scrollHeight);")

all_codes = []

for i in range(30):
    xpath = '//*[@id="page-container"]/template-base/div/div/section[1]/div/sgx-widgets-wrapper/widget-index-constituents-prices/sgx-table/div/sgx-table-list/sgx-table-row['+str(i+1)+']/sgx-table-cell-text'
    
    elem = WebDriverWait(driver,5).until(EC.element_to_be_clickable((By.XPATH,xpath)))
    elem.click()
    
    driver.execute_script("arguments[0].focus();", elem)
    driver.execute_script('arguments[0].scrollTop = arguments[0].scrollHeight', elem)
    
#     actions = ActionChains(driver)
#     actions.move_to_element(elem).perform()
    
    code = elem.text
    all_codes.append(code)
    

driver.quit()

all_codes

TimeoutException: Message: 


In [23]:
driver = webdriver.Chrome(executable_path=driver_loc)
driver.get("https://en.wikipedia.org/wiki/Straits_Times_Index")

all_codes = []

for i in range(30):
    xpath = '//*[@id="mw-content-text"]/div/table[4]/tbody/tr['+str(i+1)+']/td[1]/a[2]'
    elem = WebDriverWait(driver,10).until(EC.element_to_be_clickable((By.XPATH,xpath)))
    all_codes.append(elem.text)

driver.quit()
    
all_codes

['A17U',
 'C61U',
 'C31',
 'C38U',
 'C09',
 'C52',
 'D01',
 'D05',
 'G13',
 'H78',
 'C07',
 'J36',
 'J37',
 'BN4',
 'N2IU',
 'M44U',
 'O39',
 'S58',
 'U96',
 'C6L',
 'Z74',
 'T39',
 'S68',
 'S63',
 'Y92',
 'U11',
 'U14',
 'V03',
 'BS6',
 'F34']

In [25]:
print(sti())

Ascendas Reit (A17U) - Price: S$3.25
Latest dividend (to be) paid on 04 Mar 2020 [Rate: SGD 0.02956 Per Security]
P/B Ratio: 1.459, P/E Ratio: 20.370, EPS (TTM): 0.1560, Market Sentiment: Neutral
*As at 10 Jun 2020 05:16 PM

CapitaCom Trust (C61U) - Price: S$1.91
Latest dividend (to be) paid on 28 Feb 2020 [Rate: SGD 0.0357 Per Security]
P/B Ratio: 1.015, P/E Ratio: 16.416, EPS (TTM): 0.1140, Market Sentiment: Neutral
*As at 10 Jun 2020 05:16 PM

CapitaLand (C31) - Price: S$3.16
Latest dividend (to be) paid on 07 May 2019 [Rate: SGD 0.12 Per Security]
P/B Ratio: 0.681, P/E Ratio: 7.248, EPS (TTM): 0.4380, Market Sentiment: Neutral
*As at 10 Jun 2020 05:16 PM

CapitaMall Trust (C38U) - Price: S$2.24
Latest dividend (to be) paid on 05 Jun 2020 [Rate: SGD 0.0085 Per Security]
P/B Ratio: 1.059, P/E Ratio: 11.797, EPS (TTM): 0.1890, Market Sentiment: Neutral
*As at 10 Jun 2020 05:16 PM

CityDev (C09) - Price: S$9.10
Latest dividend (to be) paid on 16 Jul 2020 [Rate: SGD 0.08 Per Security]
P

1